<a href="https://colab.research.google.com/github/y5496694/blank-app/blob/main/shap_e/examples/sample_image_to_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. OpenAI shap-e 레포지토리 복제
!git clone https://github.com/openai/shap-e.git
%cd shap-e

# 2. editable 설치로 로컬 코드를 Python 패키지로 등록
!pip install -q -e .

# 3. 의존성 설치
!pip install -q diffusers transformers accelerate trimesh einops

# 다시 최상위 디렉토리로
%cd ..
from google.colab import files

# 업로드 위젯이 뜹니다. 여기서 2D 이미지를 골라 올리세요.
uploaded = files.upload()
# 업로드된 파일 이름을 변수에 담아 둡니다.
input_path = next(iter(uploaded.keys()))
print("✔ 업로드 완료:", input_path)
import torch
import numpy as np
from PIL import Image
from einops import rearrange

from shap_e.models.download import load_model
from shap_e.util.notebooks import (
    load_config,
    diffusion_from_config,
    decode_latent_mesh,
)

# 디바이스 설정 (GPU가 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2D→3D 인코더, 확산 모델 불러오기
xm        = load_model('transmitter', device=device)
model     = load_model('image300M',    device=device)
diffusion = diffusion_from_config(load_config('diffusion'))
print("✔ 모델 로드 완료")
# 1) 이미지 열고 [-1,1]로 정규화
img = Image.open(input_path).convert('RGB').resize((256,256))
arr = np.array(img).astype(np.float32) / 127.5 - 1.0
view = torch.from_numpy(arr)[None].permute(0,3,1,2).to(device)

# 2) 2D→3D 라텐트 인코딩
transmitted = xm.encode(view)

# 3) 확산 샘플링 (해상도 64³)
latents = diffusion.p_sample_loop(
    model_fn=lambda x, t, **kw: model(x, t, **kw),
    shape=(1, model.in_channels, 64, 64, 64),
    device=device,
    clip_denoised=True,
    model_kwargs={'transmitted_latents': transmitted}
)
print("✔ 3D 라텐트 생성 완료")
# 1) 라텐트 → 메쉬로 디코딩
mesh = decode_latent_mesh(xm, latents[0]).tri_mesh()

# 2) 파일로 저장
mesh.write_obj('output_croc.obj')
with open('output_croc.ply','wb') as f:
    mesh.write_ply(f)

print("✅ 저장 완료: output_croc.obj / output_croc.ply")
